In [16]:
!pip install -U scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, f1_score
import sklearn.metrics
import pickle
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from imblearn.under_sampling import RandomUnderSampler
data = pd.read_csv("covid.csv"); # import covid data set
X = data.drop(columns=['DATE_DIED']) # X set drops DATE_DIED
y = data['DATE_DIED'] # y set exclusively uses DATE_DIED

y = y.replace('9999-99-99', 0).replace('[^0]', 1, regex=True) # Replaces 9999-99-99 with 0, indicated no death, otherwise a 1, indicating death

undersample = RandomUnderSampler(random_state=42)

X_resampled, y_resampled = undersample.fit_resample(X, y)

X_train, X_rem, y_train, y_rem = train_test_split(X_resampled, y_resampled, train_size=0.7, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, test_size=0.5, random_state=42)

model = LGBMClassifier(boosting_type = 'gbdt', objective = 'binary', max_depth = 20, 
                       bagging_fraction = 0.6, stopping_rounds = 100, first_metric_only=False, 
                       verbose=0, min_delta=0.5)

model.num_leaves = 2^(model.max_depth)
model.num_iterations = 1000
model.early_stopping_rounds = 0.10*(model.num_iterations)
model.learning_rate = 0.05
model.fit(X_train.values, y_train)

# Validation Set Accuracy
y_pred = model.predict(X_valid)
accuracy = accuracy_score(y_valid, y_pred)

# Test Set Accuracy
y_pred_test = model.predict(X_test)
accuracy_test = accuracy_score(y_test, y_pred_test)

#True Positive Rate (Test Set)

tpr_test = recall_score(y_test, y_pred_test)

# F1 Score (precision)

f1 = f1_score(y_valid, y_pred)

# Print the accuracy score

print("Test set accuracy:", accuracy_test)

print("Validation set accuracy:", accuracy)

print("Test f1:", f1)
print("tpr:", tpr_test)

y_pred_proba = model.predict_proba(X_test)[::,1]

#calculate AUC of model
#roc_auc_score = 0
auc = sklearn.metrics.roc_auc_score(y_test, y_pred_proba)
print("AUC ", auc)


[LightGBM] [Warning] Unknown parameter: min_delta
[LightGBM] [Warning] Unknown parameter: stopping_rounds
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] Unknown parameter: min_delta
[LightGBM] [Warning] Unknown parameter: stopping_rounds
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003576 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Test set accuracy: 0.9153922800329246
Validation set accuracy: 0.9168219035653944
Test f1: 0.919779393331662
tpr: 0.9517884914463453
AUC  0.9670266241776239


In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import sklearn.metrics
import pickle
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
# loads the data set

diabetes_data = pd.read_csv("diabetes.csv")

# creates input set
    
# diabetes_data = new_diabetes_data.drop(columns=['Pregnancies'])
# diabetes_data.replace(0, np.nan, inplace=True)
# diabetes_data = impute_with_median(diabetes_data)
# diabetes_data.insert(loc=0, column='Pregnancies', value=new_diabetes_data['Pregnancies'])
# X3['Pregnancies'] = diabetes_data['Pregnancies']

# creates input set
X = diabetes_data.drop(columns=['Outcome'])
# create output set
y = diabetes_data['Outcome']

X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size=0.8)

X_valid, X_test, y_valid, y_test = train_test_split(X, y, test_size=0.5)

### MODEL TRAINING####
# creates model

model = LGBMClassifier(boosting_type = 'dart', objective = 'binary', max_depth = 100, 
                       bagging_fraction = 0.6, stopping_rounds = 100, first_metric_only=False, 
                       verbose=0, min_delta=0.5)

model.num_leaves = 2^(model.max_depth)
model.num_iterations = 1000
model.early_stopping_rounds = 0.10*(model.num_iterations)
model.learning_rate = 0.05
model.fit(X_train.values, y_train)
output = model.predict(X_test.values)

#### ACCURACY SCORE

#use model to predict probability that given y value is 1
y_pred_proba = model.predict_proba(X_test)[::,1]

#calculate AUC of model
#roc_auc_score = 0
auc = sklearn.metrics.roc_auc_score(y_test, y_pred_proba)
#### ACCURACY SCORE

score = accuracy_score(y_test, output)
# output
pickle.dump(model, open('model.pkl', 'wb'))

auc
# diabetes_data

FileNotFoundError: [Errno 2] No such file or directory: 'diabetes.csv'

In [14]:
import pandas as pd
diabetes_data = pd.read_csv("diabetes.csv")
diabetes_data

FileNotFoundError: [Errno 2] No such file or directory: 'diabetes.csv'

Context

This dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. The objective is to predict based on diagnostic measurements whether a patient has diabetes.
Content

Several constraints were placed on the selection of these instances from a larger database. In particular, all patients here are females at least 21 years old of Pima Indian heritage.

    Pregnancies: Number of times pregnant
    Glucose: Plasma glucose concentration a 2 hours in an oral glucose tolerance test
    BloodPressure: Diastolic blood pressure (mm Hg)
    SkinThickness: Triceps skin fold thickness (mm)
    Insulin: 2-Hour serum insulin (mu U/ml)
    BMI: Body mass index (weight in kg/(height in m)^2)
    DiabetesPedigreeFunction: Diabetes pedigree function
    Age: Age (years)
    Outcome: Class variable (0 or 1)

Sources:

(a) Original owners: National Institute of Diabetes and Digestive and
Kidney Diseases
(b) Donor of database: Vincent Sigillito (vgs@aplcen.apl.jhu.edu)
Research Center, RMI Group Leader
Applied Physics Laboratory
The Johns Hopkins University
Johns Hopkins Road
Laurel, MD 20707
(301) 953-6231
(c) Date received: 9 May 1990
Past Usage:

In [45]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

comp = pd.read_csv("model_comparison.csv")
comp_for_bar_plot = comp.drop(index=[6, 7 , 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])
comp_fo
sns.barplot(x='Model', y='ROC AUC', data=comp)

plt.show()


NameError: name 'comp_fo' is not defined

In [27]:
## import numpy as np
import pandas as pd
# from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import sklearn.metrics
import pickle
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
# loads the data set
diabetes_data = pd.read_csv("diabetes.csv")

# X3['Pregnancies'] = diabetes_data['Pregnancies']


# creates input set
X = diabetes_data.drop(columns=['Outcome'])
# create output set
y = diabetes_data['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

model = pickle.load(open('model.pkl', 'rb'))

#
### MODEL PREDICTION ###
output = model.predict(X_test.values)


#### ACCURACY SCORE
score = accuracy_score(y_test, output)


#use model to predict probability that given y value is 1

y_pred_proba = model.predict_proba(X_test)[::,1]

#calculate AUC of model
auc = sklearn.metrics.roc_auc_score(y_test, y_pred_proba)


#print AUC score
# auc
## pickle.dump(model, open('model.pkl', 'wb'))
### Current Accuracy - ~80% > 76%-84%

## HIT (92) AUC SCORE!!!
# sns.countplot(x='BMI',data=diabetes_data)

# sns.displot(x ='Age',kde=True,bins = 5 ,
# hue = diabetes_data['Outcome'] , palette = 'Set3', data=diabetes_data)

# sns.scatterplot(x='Glucose', y ='Insulin' ,
# data = diabetes_data , hue = 'Outcome')

# corr = diabetes_data.corr()
# sns.heatmap(corr)
auc
# sns.heatmap(corr, cmap=['blue','green', 'orange', 'red'],
# vmin = -.5 , vmax = 0.7,annot = True)

# sns.pairplot(diabetes_data)

0.9481065918653576

In [346]:
## import numpy as np
import pandas as pd
# from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import sklearn.metrics
import pickle
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from lazypredict.Supervised import LazyClassifier
# loads the data set
diabetes_data = pd.read_csv("diabetes.csv")

# creates input set
X = diabetes_data.drop(columns=['Outcome'])
# create output set
y = diabetes_data['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)


### MODEL TRAINING####
# creates model
# model = LGBMClassifier(max_depth = 100, bagging_fraction = 0.6, )

### MODEL PREDICTION ###
# output = model.predict(X_test.values)

clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

#### ACCURACY SCORE
## score = accuracy_score(y_test, output)

#use model to predict probability that given y value is 1
# # y_pred_proba = model.predict_proba(X_test)[::,1]
models
#calculate AUC of model
# # auc = sklearn.metrics.roc_auc_score(y_test, y_pred_proba)

#print AUC score

# # pickle.dump(model, open('model.pkl', 'wb'))
### Current Accuracy - ~80% > 76%-84%
# auc



100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:00<00:00, 32.51it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.82,0.79,0.79,0.82,0.07
XGBClassifier,0.83,0.78,0.78,0.83,0.07
NuSVC,0.84,0.77,0.77,0.84,0.03
SVC,0.83,0.76,0.76,0.83,0.02
BaggingClassifier,0.83,0.76,0.76,0.83,0.03
PassiveAggressiveClassifier,0.77,0.76,0.76,0.78,0.01
ExtraTreesClassifier,0.82,0.76,0.76,0.82,0.11
Perceptron,0.75,0.75,0.75,0.77,0.01
RandomForestClassifier,0.81,0.75,0.75,0.81,0.15


In [44]:
## import numpy as np
import pandas as pd
# from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import sklearn.metrics
import pickle
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
import joblib
# loads the data set
diabetes_data = pd.read_csv("diabetes.csv")

# creates input set
X = diabetes_data.drop(columns=['Outcome'])
# create output set
y = diabetes_data['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

model = pickle.load(open('model.pkl', 'rb'))

#
### MODEL PREDICTION ###
output = model.predict([[11, 210, 150, 38, 90, 28, 0.66, 49]])

## PREG, GLUCOSE, BLOOD PRESSURE, SKIN THICKNESS, INSULIN, VMI, DBF, AGE
#### ACCURACY SCORE
# score = accuracy_score(y_test, output)

#use model to predict probability that given y value is 1
# y_pred_proba = model.predict_proba(X_test)[::,1]

#calculate AUC of model
# auc = sklearn.metrics.roc_auc_score(y_test, y_pred_proba)

#print AUC score
output
## pickle.dump(model, open('model.pkl', 'wb'))
### Current Accuracy - ~80% > 76%-84%

## HIT (92) AUC SCORE!!!


array([1], dtype=int64)

In [364]:
import pandas as pd
# from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import sklearn.metrics
import pickle
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
import joblib
# loads the data set
diabetes_data = pd.read_csv("diabetes.csv")
diabetes_data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.60,0.63,50,1
1,1,85,66,29,0,26.60,0.35,31,0
2,8,183,64,0,0,23.30,0.67,32,1
3,1,89,66,23,94,28.10,0.17,21,0
4,0,137,40,35,168,43.10,2.29,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.90,0.17,63,0
764,2,122,70,27,0,36.80,0.34,27,0
765,5,121,72,23,112,26.20,0.24,30,0
766,1,126,60,0,0,30.10,0.35,47,1
